In [46]:
!pip install pyspark

In [47]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, when, count, countDistinct

In [48]:
# Initialize Spark session
spark = SparkSession.builder.appName("MySparkApp").getOrCreate()

In [49]:
charges_df = spark.read.csv("/Charges_use.csv", header=True, inferSchema=True)
primary_person_df = spark.read.csv("/Primary_Person_use.csv", header=True, inferSchema=True)
units_df = spark.read.csv("/Units_use.csv", header=True, inferSchema=True)
damages_df = spark.read.csv("/Damages_use.csv", header=True, inferSchema=True)
restrict_df = spark.read.csv("/Restrict_use.csv", header=True, inferSchema=True)

Analysing charges_df

In [50]:
#rows and columns in dataframe
num_rows = charges_df.count()
num_columns = len(charges_df.columns)

print(f"Number of Rows: {num_rows}")
print(f"Number of Columns: {num_columns}")

Number of Rows: 116110
Number of Columns: 5


In [51]:
charges_df.columns

['CRASH_ID', 'UNIT_NBR', 'PRSN_NBR', 'CHARGE', 'CITATION_NBR']

In [52]:
charges_df.printSchema()

root
 |-- CRASH_ID: integer (nullable = true)
 |-- UNIT_NBR: integer (nullable = true)
 |-- PRSN_NBR: integer (nullable = true)
 |-- CHARGE: string (nullable = true)
 |-- CITATION_NBR: string (nullable = true)



In [53]:
charges_df.head(5)

[Row(CRASH_ID=14768622, UNIT_NBR=1, PRSN_NBR=1, CHARGE='DRIVING WHILE INTOXICATED', CITATION_NBR=None),
 Row(CRASH_ID=14838637, UNIT_NBR=1, PRSN_NBR=1, CHARGE='DWI', CITATION_NBR='1600000015'),
 Row(CRASH_ID=14838641, UNIT_NBR=1, PRSN_NBR=1, CHARGE='RAN RED LIGHT SOLID (TC 544.007)', CITATION_NBR='L20440'),
 Row(CRASH_ID=14838641, UNIT_NBR=2, PRSN_NBR=1, CHARGE="NO DRIVER'S LICENSE (TC521.025)", CITATION_NBR='L23141'),
 Row(CRASH_ID=14838668, UNIT_NBR=1, PRSN_NBR=1, CHARGE='DRIVING WHILE INTOXICATED', CITATION_NBR='TX4IC50SRJD3')]

In [54]:
charges_df.describe().show(truncate=False)

+-------+--------------------+------------------+-------------------+----------------------------------+------------+
|summary|CRASH_ID            |UNIT_NBR          |PRSN_NBR           |CHARGE                            |CITATION_NBR|
+-------+--------------------+------------------+-------------------+----------------------------------+------------+
|count  |116110              |116110            |116110             |116006                            |108751      |
|mean   |1.5136505158556541E7|1.218491086039101 |1.002084230471105  |2.9604361677205883E7              |Infinity    |
|stddev |184444.05375385477  |0.5257013065906133|0.07421400061798365|5.640839947287902E7               |NaN         |
|min    |13403990            |1                 |1                  |"""ACC"" FAIL TO CONTROL SPEED"   | NO INS.    |
|max    |15738225            |12                |9                  |YURNED IMPROPERLY- WIDE RIGHT(ACC)|`38333027   |
+-------+--------------------+------------------+-------

In [55]:
null_counts = charges_df.agg(*[
    (sum(col(column).isNull().cast("int")).alias(column + "_null_count"))
    for column in charges_df.columns
])
null_counts.show(truncate=False)

+-------------------+-------------------+-------------------+-----------------+-----------------------+
|CRASH_ID_null_count|UNIT_NBR_null_count|PRSN_NBR_null_count|CHARGE_null_count|CITATION_NBR_null_count|
+-------------------+-------------------+-------------------+-----------------+-----------------------+
|0                  |0                  |0                  |104              |7359                   |
+-------------------+-------------------+-------------------+-----------------+-----------------------+



Analysing primary_person_df

In [56]:
#rows and columns in dataframe
num_rows = primary_person_df.count()
num_columns = len(primary_person_df.columns)

print(f"Number of Rows: {num_rows}")
print(f"Number of Columns: {num_columns}")

Number of Rows: 156954
Number of Columns: 32


In [57]:
primary_person_df.columns

['CRASH_ID',
 'UNIT_NBR',
 'PRSN_NBR',
 'PRSN_TYPE_ID',
 'PRSN_OCCPNT_POS_ID',
 'PRSN_INJRY_SEV_ID',
 'PRSN_AGE',
 'PRSN_ETHNICITY_ID',
 'PRSN_GNDR_ID',
 'PRSN_EJCT_ID',
 'PRSN_REST_ID',
 'PRSN_AIRBAG_ID',
 'PRSN_HELMET_ID',
 'PRSN_SOL_FL',
 'PRSN_ALC_SPEC_TYPE_ID',
 'PRSN_ALC_RSLT_ID',
 'PRSN_BAC_TEST_RSLT',
 'PRSN_DRG_SPEC_TYPE_ID',
 'PRSN_DRG_RSLT_ID',
 'DRVR_DRG_CAT_1_ID',
 'PRSN_DEATH_TIME',
 'INCAP_INJRY_CNT',
 'NONINCAP_INJRY_CNT',
 'POSS_INJRY_CNT',
 'NON_INJRY_CNT',
 'UNKN_INJRY_CNT',
 'TOT_INJRY_CNT',
 'DEATH_CNT',
 'DRVR_LIC_TYPE_ID',
 'DRVR_LIC_STATE_ID',
 'DRVR_LIC_CLS_ID',
 'DRVR_ZIP']

In [58]:
primary_person_df.printSchema()

root
 |-- CRASH_ID: integer (nullable = true)
 |-- UNIT_NBR: integer (nullable = true)
 |-- PRSN_NBR: integer (nullable = true)
 |-- PRSN_TYPE_ID: string (nullable = true)
 |-- PRSN_OCCPNT_POS_ID: string (nullable = true)
 |-- PRSN_INJRY_SEV_ID: string (nullable = true)
 |-- PRSN_AGE: string (nullable = true)
 |-- PRSN_ETHNICITY_ID: string (nullable = true)
 |-- PRSN_GNDR_ID: string (nullable = true)
 |-- PRSN_EJCT_ID: string (nullable = true)
 |-- PRSN_REST_ID: string (nullable = true)
 |-- PRSN_AIRBAG_ID: string (nullable = true)
 |-- PRSN_HELMET_ID: string (nullable = true)
 |-- PRSN_SOL_FL: string (nullable = true)
 |-- PRSN_ALC_SPEC_TYPE_ID: string (nullable = true)
 |-- PRSN_ALC_RSLT_ID: string (nullable = true)
 |-- PRSN_BAC_TEST_RSLT: string (nullable = true)
 |-- PRSN_DRG_SPEC_TYPE_ID: string (nullable = true)
 |-- PRSN_DRG_RSLT_ID: string (nullable = true)
 |-- DRVR_DRG_CAT_1_ID: string (nullable = true)
 |-- PRSN_DEATH_TIME: timestamp (nullable = true)
 |-- INCAP_INJRY_CNT: 

In [59]:
primary_person_df.head(5)

[Row(CRASH_ID=14768622, UNIT_NBR=1, PRSN_NBR=1, PRSN_TYPE_ID='DRIVER', PRSN_OCCPNT_POS_ID='FRONT LEFT', PRSN_INJRY_SEV_ID='NOT INJURED', PRSN_AGE='27', PRSN_ETHNICITY_ID='HISPANIC', PRSN_GNDR_ID='MALE', PRSN_EJCT_ID='NO', PRSN_REST_ID='NONE', PRSN_AIRBAG_ID='DEPLOYED MULTIPLE', PRSN_HELMET_ID='NOT APPLICABLE', PRSN_SOL_FL='N', PRSN_ALC_SPEC_TYPE_ID='BLOOD', PRSN_ALC_RSLT_ID='Positive', PRSN_BAC_TEST_RSLT='0.225', PRSN_DRG_SPEC_TYPE_ID='NONE', PRSN_DRG_RSLT_ID='NOT APPLICABLE', DRVR_DRG_CAT_1_ID='NOT APPLICABLE', PRSN_DEATH_TIME=None, INCAP_INJRY_CNT=0, NONINCAP_INJRY_CNT=0, POSS_INJRY_CNT=0, NON_INJRY_CNT=1, UNKN_INJRY_CNT=0, TOT_INJRY_CNT=0, DEATH_CNT=0, DRVR_LIC_TYPE_ID='DRIVER LICENSE', DRVR_LIC_STATE_ID='Texas', DRVR_LIC_CLS_ID='CLASS C', DRVR_ZIP='77357'),
 Row(CRASH_ID=14838637, UNIT_NBR=1, PRSN_NBR=1, PRSN_TYPE_ID='DRIVER', PRSN_OCCPNT_POS_ID='FRONT LEFT', PRSN_INJRY_SEV_ID='NOT INJURED', PRSN_AGE='31', PRSN_ETHNICITY_ID='WHITE', PRSN_GNDR_ID='MALE', PRSN_EJCT_ID='NO', PRSN_REST

In [60]:
primary_person_df.describe().show(truncate=False)

+-------+--------------------+------------------+--------+------------+------------------+---------------------+------------------+---------------------------+------------+------------+------------------+-----------------+----------------+-----------+---------------------+----------------+-------------------+---------------------+----------------+-----------------+--------------------+-------------------+-------------------+-------------------+--------------------+-------------------+---------------------+----------------------+-----------------+---------------+------------------+
|summary|CRASH_ID            |UNIT_NBR          |PRSN_NBR|PRSN_TYPE_ID|PRSN_OCCPNT_POS_ID|PRSN_INJRY_SEV_ID    |PRSN_AGE          |PRSN_ETHNICITY_ID          |PRSN_GNDR_ID|PRSN_EJCT_ID|PRSN_REST_ID      |PRSN_AIRBAG_ID   |PRSN_HELMET_ID  |PRSN_SOL_FL|PRSN_ALC_SPEC_TYPE_ID|PRSN_ALC_RSLT_ID|PRSN_BAC_TEST_RSLT |PRSN_DRG_SPEC_TYPE_ID|PRSN_DRG_RSLT_ID|DRVR_DRG_CAT_1_ID|INCAP_INJRY_CNT     |NONINCAP_INJRY_CNT |POSS

In [61]:
null_counts = primary_person_df.agg(*[
    (sum(col(column).isNull().cast("int")).alias(column + "_null_count"))
    for column in primary_person_df.columns
])
null_counts.show(truncate=False)

+-------------------+-------------------+-------------------+-----------------------+-----------------------------+----------------------------+-------------------+----------------------------+-----------------------+-----------------------+-----------------------+-------------------------+-------------------------+----------------------+--------------------------------+---------------------------+-----------------------------+--------------------------------+---------------------------+----------------------------+--------------------------+--------------------------+-----------------------------+-------------------------+------------------------+-------------------------+------------------------+--------------------+---------------------------+----------------------------+--------------------------+-------------------+
|CRASH_ID_null_count|UNIT_NBR_null_count|PRSN_NBR_null_count|PRSN_TYPE_ID_null_count|PRSN_OCCPNT_POS_ID_null_count|PRSN_INJRY_SEV_ID_null_count|PRSN_AGE_null_count|PRS

Analysing units_df

In [62]:
#rows and columns in dataframe
num_rows = units_df.count()
num_columns = len(units_df.columns)

print(f"Number of Rows: {num_rows}")
print(f"Number of Columns: {num_columns}")

Number of Rows: 173499
Number of Columns: 37


In [63]:
units_df.columns

['CRASH_ID',
 'UNIT_NBR',
 'UNIT_DESC_ID',
 'VEH_PARKED_FL',
 'VEH_HNR_FL',
 'VEH_LIC_STATE_ID',
 'VIN',
 'VEH_MOD_YEAR',
 'VEH_COLOR_ID',
 'VEH_MAKE_ID',
 'VEH_MOD_ID',
 'VEH_BODY_STYL_ID',
 'EMER_RESPNDR_FL',
 'OWNR_ZIP',
 'FIN_RESP_PROOF_ID',
 'FIN_RESP_TYPE_ID',
 'VEH_DMAG_AREA_1_ID',
 'VEH_DMAG_SCL_1_ID',
 'FORCE_DIR_1_ID',
 'VEH_DMAG_AREA_2_ID',
 'VEH_DMAG_SCL_2_ID',
 'FORCE_DIR_2_ID',
 'VEH_INVENTORIED_FL',
 'VEH_TRANSP_NAME',
 'VEH_TRANSP_DEST',
 'CONTRIB_FACTR_1_ID',
 'CONTRIB_FACTR_2_ID',
 'CONTRIB_FACTR_P1_ID',
 'VEH_TRVL_DIR_ID',
 'FIRST_HARM_EVT_INV_ID',
 'INCAP_INJRY_CNT',
 'NONINCAP_INJRY_CNT',
 'POSS_INJRY_CNT',
 'NON_INJRY_CNT',
 'UNKN_INJRY_CNT',
 'TOT_INJRY_CNT',
 'DEATH_CNT']

In [64]:
units_df.printSchema()

root
 |-- CRASH_ID: integer (nullable = true)
 |-- UNIT_NBR: integer (nullable = true)
 |-- UNIT_DESC_ID: string (nullable = true)
 |-- VEH_PARKED_FL: string (nullable = true)
 |-- VEH_HNR_FL: string (nullable = true)
 |-- VEH_LIC_STATE_ID: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- VEH_MOD_YEAR: string (nullable = true)
 |-- VEH_COLOR_ID: string (nullable = true)
 |-- VEH_MAKE_ID: string (nullable = true)
 |-- VEH_MOD_ID: string (nullable = true)
 |-- VEH_BODY_STYL_ID: string (nullable = true)
 |-- EMER_RESPNDR_FL: string (nullable = true)
 |-- OWNR_ZIP: string (nullable = true)
 |-- FIN_RESP_PROOF_ID: string (nullable = true)
 |-- FIN_RESP_TYPE_ID: string (nullable = true)
 |-- VEH_DMAG_AREA_1_ID: string (nullable = true)
 |-- VEH_DMAG_SCL_1_ID: string (nullable = true)
 |-- FORCE_DIR_1_ID: string (nullable = true)
 |-- VEH_DMAG_AREA_2_ID: string (nullable = true)
 |-- VEH_DMAG_SCL_2_ID: string (nullable = true)
 |-- FORCE_DIR_2_ID: string (nullable = true)
 |--

In [65]:
units_df.head(5)

[Row(CRASH_ID=14768622, UNIT_NBR=1, UNIT_DESC_ID='MOTOR VEHICLE', VEH_PARKED_FL='N', VEH_HNR_FL='N', VEH_LIC_STATE_ID='TX', VIN='4S2CK57D1X4381118', VEH_MOD_YEAR='1999', VEH_COLOR_ID='GRY', VEH_MAKE_ID='ISUZU', VEH_MOD_ID='AMIGO', VEH_BODY_STYL_ID='PASSENGER CAR, 4-DOOR', EMER_RESPNDR_FL='Y', OWNR_ZIP='77357', FIN_RESP_PROOF_ID='2', FIN_RESP_TYPE_ID='NA', VEH_DMAG_AREA_1_ID='FRONT END DAMAGE CONCENTRATED IMPACT', VEH_DMAG_SCL_1_ID='DAMAGED 3', FORCE_DIR_1_ID='1', VEH_DMAG_AREA_2_ID='LEFT SIDE AND TOP DAMAGE ROLLOVER EFFECTS', VEH_DMAG_SCL_2_ID='DAMAGED 4', FORCE_DIR_2_ID='3', VEH_INVENTORIED_FL='Y', VEH_TRANSP_NAME='VALENTIN TOWING', VEH_TRANSP_DEST='20288 FM 1314 PORTER, TX', CONTRIB_FACTR_1_ID='UNDER INFLUENCE - ALCOHOL', CONTRIB_FACTR_2_ID='NA', CONTRIB_FACTR_P1_ID='NA', VEH_TRVL_DIR_ID='EAST', FIRST_HARM_EVT_INV_ID='Y', INCAP_INJRY_CNT=0, NONINCAP_INJRY_CNT=0, POSS_INJRY_CNT=0, NON_INJRY_CNT=1, UNKN_INJRY_CNT=0, TOT_INJRY_CNT=0, DEATH_CNT=0),
 Row(CRASH_ID=14838637, UNIT_NBR=1, UNI

In [66]:
units_df.describe().show(truncate=False)

+-------+--------------------+------------------+-------------+-------------+----------+----------------+---------------------+-----------------+-------------------+-----------+-----------------+-----------------+---------------+-------------------+------------------+---------------------------------------+--------------------------------------+-----------------+-----------------+--------------------------------------+-----------------+-----------------+------------------+-------------------------+----------------------------+------------------------+------------------------+------------------------+---------------+---------------------+--------------------+-------------------+-------------------+------------------+--------------------+-------------------+---------------------+
|summary|CRASH_ID            |UNIT_NBR          |UNIT_DESC_ID |VEH_PARKED_FL|VEH_HNR_FL|VEH_LIC_STATE_ID|VIN                  |VEH_MOD_YEAR     |VEH_COLOR_ID       |VEH_MAKE_ID|VEH_MOD_ID       |VEH_BODY_STYL_ID

In [67]:
null_counts = units_df.agg(*[
    (sum(col(column).isNull().cast("int")).alias(column + "_null_count"))
    for column in units_df.columns
])
null_counts.show(truncate=False)

+-------------------+-------------------+-----------------------+------------------------+---------------------+---------------------------+--------------+-----------------------+-----------------------+----------------------+---------------------+---------------------------+--------------------------+-------------------+----------------------------+---------------------------+-----------------------------+----------------------------+-------------------------+-----------------------------+----------------------------+-------------------------+-----------------------------+--------------------------+--------------------------+-----------------------------+-----------------------------+------------------------------+--------------------------+--------------------------------+--------------------------+-----------------------------+-------------------------+------------------------+-------------------------+------------------------+--------------------+
|CRASH_ID_null_count|UNIT_NBR_null

In [80]:
units_df.select("VEH_COLOR_ID").distinct().show(100, truncate = False)

+------------+
|VEH_COLOR_ID|
+------------+
|RED         |
|TEA         |
|BLK         |
|ONG         |
|YEL         |
|GLD         |
|NA          |
|98          |
|99          |
|CAM         |
|BRZ         |
|TRQ         |
|SIL         |
|CPR         |
|BRO         |
|PLE         |
|BLU         |
|MAR         |
|BGE         |
|PNK         |
|WHI         |
|TAN         |
|MUL         |
|GRN         |
|GRY         |
+------------+



In [81]:
units_df.groupby("VEH_MAKE_ID").count().orderBy(col("count").desc()).show()

+------------+-----+
| VEH_MAKE_ID|count|
+------------+-----+
|        FORD|29792|
|   CHEVROLET|28073|
|      TOYOTA|16487|
|       DODGE|13091|
|      NISSAN|10964|
|       HONDA|10460|
|          NA| 6234|
|         GMC| 5044|
|        JEEP| 4170|
|     HYUNDAI| 3859|
|         KIA| 3186|
|    CHRYSLER| 3144|
|FREIGHTLINER| 2861|
|       MAZDA| 2585|
|     PONTIAC| 2144|
|  VOLKSWAGEN| 2141|
|       LEXUS| 2087|
|       BUICK| 1965|
|    CADILLAC| 1796|
|  MITSUBISHI| 1756|
+------------+-----+
only showing top 20 rows



In [82]:
units_df.filter(col("VEH_LIC_STATE_ID").cast("int").isNull()).groupby("VEH_LIC_STATE_ID").count().orderBy(col("count").desc()).show(100)


+----------------+------+
|VEH_LIC_STATE_ID| count|
+----------------+------+
|              TX|156997|
|              NA|  6065|
|              UN|  1791|
|              OK|   970|
|              LA|   782|
|              NM|   731|
|              IN|   544|
|              MX|   493|
|              CA|   482|
|              FL|   416|
|              IL|   370|
|              AR|   350|
|              TN|   304|
|              MS|   244|
|              AZ|   236|
|              KS|   186|
|              MO|   177|
|              GA|   177|
|              CO|   175|
|              NC|   159|
|              AL|   145|
|              OH|   127|
|              MI|   122|
|              MN|   113|
|              WI|   101|
|              VA|    98|
|              NE|    88|
|              WA|    85|
|              IA|    79|
|              UT|    75|
|              OR|    71|
|              PA|    70|
|              SC|    63|
|              CD|    57|
|              NY|    50|
|           

Analysing damages_df

In [68]:
#rows and columns in dataframe
num_rows = damages_df.count()
num_columns = len(damages_df.columns)

print(f"Number of Rows: {num_rows}")
print(f"Number of Columns: {num_columns}")

Number of Rows: 24950
Number of Columns: 2


In [69]:
damages_df.columns

['CRASH_ID', 'DAMAGED_PROPERTY']

In [70]:
damages_df.printSchema()

root
 |-- CRASH_ID: integer (nullable = true)
 |-- DAMAGED_PROPERTY: string (nullable = true)



In [71]:
damages_df.head(5)

[Row(CRASH_ID=14768622, DAMAGED_PROPERTY='MAILBOX'),
 Row(CRASH_ID=14768622, DAMAGED_PROPERTY='YARD, GRASS'),
 Row(CRASH_ID=14838668, DAMAGED_PROPERTY='GUARDRAIL'),
 Row(CRASH_ID=14838685, DAMAGED_PROPERTY='ROAD SIGN'),
 Row(CRASH_ID=14838693, DAMAGED_PROPERTY='2009 MAZDA 3')]

In [72]:
damages_df.describe().show(truncate=False)

+-------+--------------------+---------------------------+
|summary|CRASH_ID            |DAMAGED_PROPERTY           |
+-------+--------------------+---------------------------+
|count  |24950               |24944                      |
|mean   |1.5172638111623246E7|1303475.0                  |
|stddev |194882.21908891975  |NULL                       |
|min    |13403990            |"""BUS STOP AHEAD"" SIGN"  |
|max    |15717920            |YOUR STORAGE PLACE BUILDING|
+-------+--------------------+---------------------------+



In [73]:
null_counts = damages_df.agg(*[
    (sum(col(column).isNull().cast("int")).alias(column + "_null_count"))
    for column in damages_df.columns
])
null_counts.show(truncate=False)

+-------------------+---------------------------+
|CRASH_ID_null_count|DAMAGED_PROPERTY_null_count|
+-------------------+---------------------------+
|0                  |6                          |
+-------------------+---------------------------+



Analysing restrict_df

In [74]:
#rows and columns in dataframe
num_rows = restrict_df.count()
num_columns = len(restrict_df.columns)

print(f"Number of Rows: {num_rows}")
print(f"Number of Columns: {num_columns}")

Number of Rows: 159852
Number of Columns: 3


In [75]:
restrict_df.columns

['CRASH_ID', 'UNIT_NBR', 'DRVR_LIC_RESTRIC_ID']

In [76]:
restrict_df.printSchema()

root
 |-- CRASH_ID: integer (nullable = true)
 |-- UNIT_NBR: integer (nullable = true)
 |-- DRVR_LIC_RESTRIC_ID: string (nullable = true)



In [77]:
restrict_df.head(5)

[Row(CRASH_ID=14768622, UNIT_NBR=1, DRVR_LIC_RESTRIC_ID='NONE'),
 Row(CRASH_ID=14838637, UNIT_NBR=1, DRVR_LIC_RESTRIC_ID='OTHER/OUT OF STATE'),
 Row(CRASH_ID=14838637, UNIT_NBR=2, DRVR_LIC_RESTRIC_ID='NONE'),
 Row(CRASH_ID=14838641, UNIT_NBR=1, DRVR_LIC_RESTRIC_ID='WITH CORRECTIVE LENSES'),
 Row(CRASH_ID=14838641, UNIT_NBR=2, DRVR_LIC_RESTRIC_ID='UNLICENSED')]

In [78]:
restrict_df.describe().show(truncate=False)

+-------+--------------------+------------------+-----------------------------+
|summary|CRASH_ID            |UNIT_NBR          |DRVR_LIC_RESTRIC_ID          |
+-------+--------------------+------------------+-----------------------------+
|count  |159852              |159852            |159852                       |
|mean   |1.5131566613286039E7|1.5918975051923028|NULL                         |
|stddev |182427.6014999234   |0.768452230763807 |NULL                         |
|min    |13403990            |1                 |APPLICABLE PROSTHETIC DEVICES|
|max    |15738225            |14                |WITH TELESCOPIC LENS         |
+-------+--------------------+------------------+-----------------------------+



In [79]:
null_counts = restrict_df.agg(*[
    (sum(col(column).isNull().cast("int")).alias(column + "_null_count"))
    for column in restrict_df.columns
])
null_counts.show(truncate=False)

+-------------------+-------------------+------------------------------+
|CRASH_ID_null_count|UNIT_NBR_null_count|DRVR_LIC_RESTRIC_ID_null_count|
+-------------------+-------------------+------------------------------+
|0                  |0                  |0                             |
+-------------------+-------------------+------------------------------+

